# Convex Hulls and Delaunay Triangulation
- Computational geometry algorithms
- Real examples: Shape analysis, Mesh generation

In [1]:
import numpy as np
from scipy.spatial import ConvexHull, Delaunay, Voronoi
import matplotlib.pyplot as plt
print('Computational geometry module loaded')

Computational geometry module loaded


## Convex Hull
**Definition**: Smallest convex polygon containing all points

**Properties**:
- vertices: Points on hull
- simplices: Edges (2D) or faces (3D)
- volume/area: Enclosed space

**Applications**: Shape boundary, outlier detection, collision

In [2]:
# Random 2D points
np.random.seed(42)
points = np.random.rand(30, 2)

print(f'Computing convex hull for {len(points)} points\n')

# Convex hull
hull = ConvexHull(points)

print(f'Convex hull properties:')
print(f'  Vertices: {len(hull.vertices)} points')
print(f'  Edges: {len(hull.simplices)}')
print(f'  Area: {hull.volume:.4f}')  # 'volume' is area in 2D
print(f'  Perimeter: {hull.area:.4f}\n')  # 'area' is perimeter in 2D

print(f'Hull vertices (indices): {hull.vertices}')

Computing convex hull for 30 points

Convex hull properties:
  Vertices: 11 points
  Edges: 11
  Area: 0.7091
  Perimeter: 3.2289

Hull vertices (indices): [ 6 25 17 26  0  5 29 28  2 18 14]


## Delaunay Triangulation
**Definition**: Triangulation maximizing minimum angle

**Properties**:
- No point inside circumcircle of any triangle
- Unique for non-degenerate point sets
- Dual of Voronoi diagram

**Applications**: Mesh generation, interpolation, graphics

In [3]:
# Delaunay triangulation
print('Computing Delaunay triangulation\n')

tri = Delaunay(points)

print(f'Triangulation properties:')
print(f'  Points: {tri.npoints}')
print(f'  Triangles: {len(tri.simplices)}')
print(f'  First triangle vertices: {tri.simplices[0]}\n')

# Check if point is inside triangulation
test_point = np.array([0.5, 0.5])
simplex_id = tri.find_simplex(test_point)
if simplex_id >= 0:
    print(f'Point {test_point} is inside triangle {simplex_id}')
else:
    print(f'Point {test_point} is outside triangulation')

Computing Delaunay triangulation

Triangulation properties:
  Points: 30
  Triangles: 47
  First triangle vertices: [19  4 23]

Point [0.5 0.5] is inside triangle 0


## Real Example: GPS Coverage Area
Compute service area from tower locations
Convex hull = boundary of coverage

In [4]:
# Cell tower locations (km coordinates)
np.random.seed(42)
tower_locs = np.random.rand(15, 2) * 50

print('Cell Tower Coverage Analysis')
print(f'  Number of towers: {len(tower_locs)}\n')

# Coverage area (convex hull)
hull = ConvexHull(tower_locs)

print(f'Coverage area:')
print(f'  Area: {hull.volume:.2f} km²')
print(f'  Perimeter: {hull.area:.2f} km')
print(f'  Border towers: {len(hull.vertices)}\n')

# Identify border towers (need reinforcement)
border_towers = hull.vertices
print(f'Border tower IDs: {border_towers}')
print('These towers define service boundary')

Cell Tower Coverage Analysis
  Number of towers: 15

Coverage area:
  Area: 1265.20 km²
  Perimeter: 141.11 km
  Border towers: 6

Border tower IDs: [14  6  1  0  5  2]
These towers define service boundary


## Real Example: Terrain Mesh Generation
Create triangular mesh from elevation points
Delaunay for smooth interpolation

In [5]:
# Elevation survey points (x, y, elevation)
np.random.seed(42)
n_points = 50
xy = np.random.rand(n_points, 2) * 100  # km
elevations = 500 + 200*np.sin(xy[:, 0]/20) + 100*np.cos(xy[:, 1]/15)

print('Terrain Mesh Generation')
print(f'  Survey points: {n_points}')
print(f'  Area: 100x100 km²\n')

# Delaunay triangulation
tri = Delaunay(xy)

print(f'Generated mesh:')
print(f'  Triangles: {len(tri.simplices)}')
print(f'  Edges per triangle: 3')
print(f'  Total vertices: {tri.npoints}\n')

# Interpolate elevation at new point
new_point = np.array([[50, 50]])
simplex = tri.find_simplex(new_point)

if simplex >= 0:
    # Get triangle vertices
    triangle_verts = tri.simplices[simplex]
    # Barycentric coordinates for interpolation
    b = tri.transform[simplex, :2].dot(new_point.T - tri.transform[simplex, 2].reshape(2,1))
    b = np.r_[b, 1 - b.sum(axis=0)]
    # Interpolate
    interp_elevation = np.dot(elevations[triangle_verts], b.flatten())
    print(f'Interpolated elevation at {new_point[0]}:')
    print(f'  {interp_elevation:.1f} meters')

Terrain Mesh Generation
  Survey points: 50
  Area: 100x100 km²

Generated mesh:
  Triangles: 87
  Edges per triangle: 3
  Total vertices: 50



ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

## Voronoi Diagram
**Definition**: Partitions space into regions
Each region = points closest to one seed

**Dual of Delaunay**: Edges perpendicular to Delaunay edges

**Applications**: Facility location, biology (cell territories)

In [ ]:
# Voronoi diagram
vor = Voronoi(points)

print('Voronoi Diagram')
print(f'  Input points: {len(vor.points)}')
print(f'  Voronoi vertices: {len(vor.vertices)}')
print(f'  Regions: {len(vor.regions)}\n')

print('Each region = area closest to one point')
print('Applications:')
print('  - Nearest facility assignment')
print('  - Territory boundaries')
print('  - Natural neighbor interpolation')

## Summary

### Computational Geometry Functions:
```python
from scipy.spatial import ConvexHull, Delaunay, Voronoi

# Convex hull
hull = ConvexHull(points)
area = hull.volume  # 2D area
perimeter = hull.area  # 2D perimeter

# Delaunay triangulation
tri = Delaunay(points)
triangles = tri.simplices
simplex_id = tri.find_simplex(query_point)

# Voronoi
vor = Voronoi(points)
regions = vor.regions
```

### Applications:

**Convex Hull**:
- Shape boundary detection
- Outlier detection (points far from hull)
- Collision detection
- Gift wrapping problems

**Delaunay Triangulation**:
- Mesh generation (FEM, graphics)
- Terrain modeling
- Interpolation (natural neighbor)
- Path planning

**Voronoi Diagram**:
- Facility location (nearest hospital)
- Territory assignment
- Cell growth simulation
- Nearest neighbor classification

### Best Practices:
✓ **Check dimensions**: Works in any dimension  
✓ **Degenerate cases**: Colinear/coplanar points  
✓ **Memory**: O(n^(d/2)) for d dimensions  
✓ **Visualization**: matplotlib for 2D, mayavi for 3D  